# Import Required Libraries
Import the necessary libraries, including pandas, numpy, matplotlib, statsmodels, sklearn, and Prophet.

In [ ]:
# Import the necessary libraries
import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical operations
import matplotlib.pyplot as plt  # For plotting graphs
import statsmodels.api as sm  # For statistical models
from sklearn.metrics import mean_squared_error  # For calculating mean squared error
from prophet import Prophet  # For time series forecasting

# Load and Clean Data
Load the CSV files, rename columns, drop rows with all NaN values, clean numeric columns, and convert the 'Period' column to datetime.

In [ ]:
# List of file paths to be processed
files = [
    r'C:\Users\yagiz\OneDrive\Masaüstü\Uygulamalar\kodlar\insurance\Kasko 2019 2023.csv',
    r'C:\Users\yagiz\OneDrive\Masaüstü\Uygulamalar\kodlar\insurance\Kasko_2013-2018.csv'
]

# Function to clean and prepare each file
def clean_file(file_path):
    # Load the file
    df = pd.read_csv(file_path)

    # Rename columns for clarity
    df.columns = [
        "Period",
        "Gross_Written_Premium_TL",
        "Gross_Increase_Rate",
        "Net_Written_Premium_TL",
        "Net_Increase_Rate",
        "Net_Earned_Premium_TL",
        "Earned_Increase_Rate",
        "Net_Realized_Loss_TL",
        "Loss_Increase_Rate",
        "Technical_Profit_TL",
        "Technical_Profit_Increase_Rate",
        "Net_Loss_Premium_Rate",
        "Combined_Ratio"
    ]

    # Drop rows with all NaN values
    df = df.dropna(how='all').reset_index(drop=True)

    # Clean numeric columns
    numeric_columns = [
        "Gross_Written_Premium_TL",
        "Gross_Increase_Rate",
        "Net_Written_Premium_TL",
        "Net_Increase_Rate",
        "Net_Earned_Premium_TL",
        "Earned_Increase_Rate",
        "Net_Realized_Loss_TL",
        "Loss_Increase_Rate",
        "Technical_Profit_TL",
        "Technical_Profit_Increase_Rate",
        "Net_Loss_Premium_Rate",
        "Combined_Ratio"
    ]

    for col in numeric_columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace(r"[^\d.,()-]", "", regex=True)  # Remove non-numeric characters
            .str.replace(",", "", regex=False)  # Remove commas
            .str.replace(r"\((.*?)\)", r"-\1", regex=True)  # Convert parentheses to negative sign
        )
        df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert to numeric

    # Convert "Period" to datetime
    df["Period"] = pd.to_datetime(df["Period"], format='%Y', errors='coerce')

    return df

# Process and combine all files
dataframes = [clean_file(file) for file in files]
combined_data = pd.concat(dataframes, ignore_index=True)

# Set "Period" as the index and sort
combined_data = combined_data.set_index("Period").sort_index()

# Fill missing values (forward fill)
combined_data = combined_data.ffill()

# Ensure no NaN or infinite values
combined_data = combined_data.replace([np.inf, -np.inf], np.nan).dropna()

# Remove rows where the index is NaT
combined_data = combined_data[combined_data.index.notna()]

combined_data.head()  # Display the first few rows of the cleaned data

# Combine Data from Multiple Files
Combine the cleaned data from multiple files into a single DataFrame, set 'Period' as the index, sort the index, and fill missing values.

In [ ]:
# Combine Data from Multiple Files

# Process and combine all files
dataframes = [clean_file(file) for file in files]

# Concatenate all dataframes into a single dataframe
combined_data = pd.concat(dataframes, ignore_index=True)

# Set "Period" as the index and sort the index
combined_data = combined_data.set_index("Period").sort_index()

# Fill missing values using forward fill method
combined_data = combined_data.ffill()

# Replace infinite values with NaN and drop rows with NaN values
combined_data = combined_data.replace([np.inf, -np.inf], np.nan).dropna()

# Remove rows where the index is NaT (Not a Time)
combined_data = combined_data[combined_data.index.notna()]

# Display the first few rows of the combined data
combined_data.head()

# Plot Initial Data
Plot the initial data to visualize the Gross Written Premium, Net Written Premium, and Technical Profit over the years.

In [ ]:
# Plot Initial Data

# Plot the initial data to visualize the Gross Written Premium, Net Written Premium, and Technical Profit over the years.
plt.figure(figsize=(14, 7))

# Plot Gross Written Premium
plt.plot(combined_data.index, combined_data['Gross_Written_Premium_TL'] / 10**9, label='Gross Written Premium TL (in billions)')

# Plot Net Written Premium
plt.plot(combined_data.index, combined_data['Net_Written_Premium_TL'] / 10**9, label='Net Written Premium TL (in billions)')

# Plot Technical Profit
plt.plot(combined_data.index, combined_data['Technical_Profit_TL'] / 10**9, label='Technical Profit TL (in billions)')

# Add labels and title
plt.xlabel('Year')
plt.ylabel('Amount (in billions TL)')
plt.title('Yearly Gross Written Premium, Net Written Premium, and Technical Profit')

# Add legend
plt.legend()

# Show plot
plt.show()

# Prepare Data for Prophet
Prepare the data for Prophet by resetting the index and renaming columns to 'ds' and 'y'.

In [ ]:
# Prepare Data for Prophet

# Reset the index and rename columns to 'ds' and 'y' for Prophet
prophet_data = combined_data.reset_index()[['Period', 'Technical_Profit_TL']]
prophet_data.columns = ['ds', 'y']

# Display the first few rows of the prepared data for Prophet
prophet_data.head()

# Fit Prophet Model
Initialize and fit the Prophet model using the prepared data.

In [ ]:
# Fit Prophet Model

# Initialize the Prophet model
model = Prophet()

# Fit the model with the prepared data
model.fit(prophet_data)

# Create a dataframe for future dates (5 years into the future)
future = model.make_future_dataframe(periods=5*12, freq='MS')

# Predict future values
forecast = model.predict(future)

# Plot the forecast
fig = model.plot(forecast)
plt.title('Technical Profit Forecast with Uncertainty Intervals')
plt.xlabel('Year')
plt.ylabel('Technical Profit TL (in billions)')
plt.show()

# Plot the forecast components
fig2 = model.plot_components(forecast)
plt.show()

# Forecast Future Data
Create a future DataFrame for 5 years, make predictions using the Prophet model, and plot the forecast results.

In [ ]:
# Forecast Future Data

# Create a future DataFrame for 5 years, make predictions using the Prophet model, and plot the forecast results.

# Create a dataframe for future dates (5 years into the future)
future = model.make_future_dataframe(periods=5*12, freq='MS')

# Predict future values
forecast = model.predict(future)

# Plot the forecast
fig = model.plot(forecast)
plt.title('Technical Profit Forecast with Uncertainty Intervals')
plt.xlabel('Year')
plt.ylabel('Technical Profit TL (in billions)')
plt.show()

# Plot the forecast components
fig2 = model.plot_components(forecast)
plt.show()

# Plot the forecasted data in billions
plt.figure(figsize=(14, 7))
plt.plot(forecast['ds'], forecast['yhat'] / 10**9, label='Forecasted Technical Profit TL (in billions)')
plt.fill_between(forecast['ds'], forecast['yhat_lower'] / 10**9, forecast['yhat_upper'] / 10**9, color='gray', alpha=0.2, label='Uncertainty Interval')
plt.xlabel('Year')
plt.ylabel('Technical Profit TL (in billions)')
plt.title('Forecasted Technical Profit with Uncertainty Intervals')
plt.legend()
plt.show()

# Plot Forecast Results
Plot the forecasted Technical Profit with uncertainty intervals and the forecast components.

In [ ]:
# Plot Forecast Results

# Plot the forecasted data in billions
plt.figure(figsize=(14, 7))
plt.plot(forecast['ds'], forecast['yhat'] / 10**9, label='Forecasted Technical Profit TL (in billions)')
plt.fill_between(forecast['ds'], forecast['yhat_lower'] / 10**9, forecast['yhat_upper'] / 10**9, color='gray', alpha=0.2, label='Uncertainty Interval')
plt.xlabel('Year')
plt.ylabel('Technical Profit TL (in billions)')
plt.title('Forecasted Technical Profit with Uncertainty Intervals')
plt.legend()
plt.show()

# Plot the forecast components
fig2 = model.plot_components(forecast)
plt.show()

# Calculate and Plot Mean Squared Error (MSE)
Calculate the Mean Squared Error (MSE) between the actual and predicted Technical Profit, and plot the actual vs predicted data with the MSE.

In [ ]:
# Calculate and Plot Mean Squared Error (MSE)

# Ensure the actual data and forecast data are aligned
actual = prophet_data.set_index('ds')['y']
predicted = forecast.set_index('ds')['yhat']
common_dates = actual.index.intersection(predicted.index)
actual_common = actual.loc[common_dates]
predicted_common = predicted.loc[common_dates]

# Ensure the lengths are consistent
if len(actual_common) > len(predicted_common):
    actual_common = actual_common.iloc[:len(predicted_common)]
elif len(predicted_common) > len(actual_common):
    predicted_common = predicted_common.iloc[:len(actual_common)]

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(actual_common, predicted_common)

# Plot the actual vs predicted data with MSE
plt.figure(figsize=(14, 7))
plt.plot(common_dates, actual_common / 10**9, label='Actual Technical Profit TL (in billions)')
plt.plot(common_dates, predicted_common / 10**9, label='Predicted Technical Profit TL (in billions)')
plt.xlabel('Year')
plt.ylabel('Technical Profit TL (in billions)')
plt.title(f'Mean Squared Error: {mse:.2f}')
plt.legend()
plt.show()